In [1]:
%config InlineBackend.rc = {"figure.dpi": 72, 'figure.figsize': (6.0, 4.0)}
%matplotlib ipympl

import ase 
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import HTML

import abtem

abtem.config.set({"local_diagnostics.progress_bar": True})
abtem.config.set({"device": "cpu"});

(examples:interactions)=
# Interactive visualizations

## STEM

In [2]:
atoms = ase.build.mx2(vacuum=3)
atoms = abtem.orthogonalize_cell(atoms) * (3, 2, 1)

probe = abtem.Probe(
    semiangle_cutoff=np.linspace(20, 30, 3),
    defocus=np.linspace(0, 50, 3),
    Cs=np.linspace(0, 5e5, 3),
    astigmatism_angle=np.pi / 4,
    sampling=0.05,
    energy=80e3,
)

potential = abtem.Potential(atoms, projection="infinite", slice_thickness=2)

scan = abtem.GridScan(
    start=(0, 0),
    end=(1 / 3, 1 / 2),
    fractional=True,
    potential=potential,
)

doses = np.geomspace(1e1, 1e7, 30)

measurements = (
    probe.scan(potential, scan=scan)
    .to_image_ensemble()
    .gaussian_filter(0.3)
    .interpolate(0.1)
    .tile((3, 2))
    .poisson_noise(dose_per_area=doses)
    .compute()
)

[########################################] | 100% Completed | 76.23 s


In [19]:
visualization = measurements.show(interact=True, cbar=True)

In [4]:
animated_measurment = measurements[:, -1, 0, 0, 50:150].sum(1)

visualization = animated_measurment.show(display=False)

visualization.axis_off(spines=False)
visualization.adjust_tight_bbox()
animation = visualization.animate(adjust_scale=True, interval=10)

In [6]:
animation.save("../thumbnails/interactions.gif", writer="pillow")